In [1]:
import torch
import numpy as np
import zarr
import os
import pandas as pd
import dotenv
dotenv.load_dotenv()

True

In [2]:
store = zarr.DirectoryStore(os.environ.get('DATAFOLDER'))
sources = zarr.group(store=store)

In [3]:
def stunden_zu_datum(stunden_array):
    basis_datum = pd.Timestamp('1959-01-01 00:00:00')
    datum_array = [basis_datum + pd.Timedelta(hours=int(h)) for h in stunden_array]
    return datum_array

times = np.array(sources["time"])
times = stunden_zu_datum(times)

times_gt = times >= np.datetime64("1993-01-01T00:00:00")
times_ls = times <= np.datetime64("2016-12-31T23:59:59")
times = times_gt & times_ls
idxs = np.arange(sources["time"].shape[0])[times]

In [4]:
#idxs = np.arange(10)
mins = torch.Tensor(
            [193.48901, -3.3835982e-05, -65.45247, -96.98215, -6838.8906]
)
maxs = torch.Tensor(
            [324.80637, 0.029175894, 113.785934, 89.834595, 109541.625]
)

max_minus_min = maxs - mins
mins = mins[:, None, None]
max_minus_min = max_minus_min[:, None, None]

In [5]:
for env_var in ["temperature", "specific_humidity", "u_component_of_wind", "v_component_of_wind",
                         "geopotential"]:
    tensor = sources[env_var][idxs]
    tensor = torch.Tensor(np.mean(tensor[:,[0, 3, 6, 9, 12],:,:], axis=0)).permute((0,2,1))
    tensor = (tensor - mins) / max_minus_min
    torch.save(tensor, f'./{env_var}.pt')
    print(tensor[0,0,:10])
    print(tensor.shape)

tensor([0.1026, 0.1026, 0.1026, 0.1026, 0.1026, 0.1026, 0.1026, 0.1026, 0.1026,
        0.1026])
torch.Size([5, 121, 240])
tensor([-1.4734, -1.4734, -1.4734, -1.4734, -1.4734, -1.4734, -1.4734, -1.4734,
        -1.4734, -1.4734])
torch.Size([5, 121, 240])
tensor([-1.4932, -1.4937, -1.4942, -1.4947, -1.4951, -1.4956, -1.4960, -1.4964,
        -1.4968, -1.4971])
torch.Size([5, 121, 240])
tensor([-1.4537, -1.4543, -1.4550, -1.4556, -1.4562, -1.4569, -1.4576, -1.4583,
        -1.4589, -1.4596])
torch.Size([5, 121, 240])
tensor([1435.9841, 1435.9790, 1435.9758, 1435.9725, 1435.9675, 1435.9630,
        1435.9596, 1435.9547, 1435.9513, 1435.9478])
torch.Size([5, 121, 240])
